# Setup

In [5]:
import json
import pandas as pd
import numpy as np
import os
import boto3

In [3]:
AWS_ACCESS_KEY_ID = os.environ["AWS_ACCESS_KEY_ID"]
AWS_SECRET_ACCESS_KEY = os.environ["AWS_SECRET_ACCESS_KEY"]
AWS_DEFAULT_REGION = os.environ["AWS_DEFAULT_REGION"]

In [2]:
os.environ['AWS_DEFAULT_REGION']

'eu-west-3'

# Data exploration

In [8]:
with open("Recipes.json", "r") as f:
    ds = json.load(f)
ds.keys()

dict_keys(['Items', 'Count', 'ScannedCount', 'ConsumedCapacity'])

In [17]:
ds["Count"]

8

In [14]:
ds["Items"][0].keys()

dict_keys(['date', 'nutriScore', 'id', 'data'])

In [16]:
ds["Items"][0]["nutriScore"]

{'N': '-297529017418066210'}

In [38]:
first_recipe = ds["Items"][0]["data"]['M']
first_recipe.keys()

dict_keys(['mealType', 'ingredients', 'title', 'numServings', 'cookingMethod', 'nutritionTable'])

In [25]:
first_recipe["mealType"]

{'S': 'Snack'}

In [35]:
first_recipe["cookingMethod"]

{'S': 'Add all ingredients to a blender and\nblend on full power.'}

In [46]:
first_recipe["nutritionTable"]['M']

{'kcal': {'M': {'percentOfDailyNeeds': {'N': '17.82'},
   'amount': {'N': '356.4'},
   'unit': {'S': ''}}},
 'sodium': {'M': {'percentOfDailyNeeds': {'N': '6.12'},
   'amount': {'N': '140.72'},
   'unit': {'S': 'mg'}}},
 'sugars': {'M': {'percentOfDailyNeeds': {'N': '28.82'},
   'amount': {'N': '25.94'},
   'unit': {'S': 'g'}}},
 'carbs': {'M': {'percentOfDailyNeeds': {'N': '12.58'},
   'amount': {'N': '34.59'},
   'unit': {'S': 'g'}}},
 'protein': {'M': {'percentOfDailyNeeds': {'N': '68.43'},
   'amount': {'N': '34.21'},
   'unit': {'S': 'g'}}},
 'fat': {'M': {'percentOfDailyNeeds': {'N': '15.63'},
   'amount': {'N': '10.16'},
   'unit': {'S': 'g'}}},
 'saturates': {'M': {'percentOfDailyNeeds': {'N': '32.21'},
   'amount': {'N': '5.15'},
   'unit': {'S': 'g'}}},
 'fibre': {'M': {'percentOfDailyNeeds': {'N': '10.4'},
   'amount': {'N': '2.6'},
   'unit': {'S': 'g'}}}}

In [37]:
pd.DataFrame.from_dict(first_recipe["nutritionTable"]['M'])

,kcal,sodium,sugars,carbs,protein,fat,saturates,fibre
M,"{'percentOfDailyNeeds': {'N': '17.82'}, 'amoun...","{'percentOfDailyNeeds': {'N': '6.12'}, 'amount...","{'percentOfDailyNeeds': {'N': '28.82'}, 'amoun...","{'percentOfDailyNeeds': {'N': '12.58'}, 'amoun...","{'percentOfDailyNeeds': {'N': '68.43'}, 'amoun...","{'percentOfDailyNeeds': {'N': '15.63'}, 'amoun...","{'percentOfDailyNeeds': {'N': '32.21'}, 'amoun...","{'percentOfDailyNeeds': {'N': '10.4'}, 'amount..."


In [34]:
possible_mealType = set()
for recipe in ds["Items"]:
    possible_mealType.add(recipe["data"]['M']["mealType"]['S'])
possible_mealType

{'Breakfast', 'Dinner', 'Lunch', 'Snack'}

In [47]:
possible_units = set()
for recipe in ds["Items"]:
    nut_tab = recipe["data"]['M']["nutritionTable"]['M']
    for nutrition in nut_tab.values():
        possible_units.add(nutrition['M']['unit']['S'])
possible_units

{'', 'g', 'mg'}

# Upload data to DynamoDB

In [ ]:
# Create table
# !aws dynamodb create-table --cli-input-json file://Recipes_schema.json

In [9]:
dynamoclient = boto3.client('dynamodb', region_name=AWS_DEFAULT_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
targettabname = "Recipes"

In [ ]:
# Upload Recipes.json -> Recipes
for item in ds["Items"]:
    response = dynamoclient.put_item(
        TableName=targettabname,
        Item=item
    )